# Library

In [1]:
import pandas as pd
import numpy as np
import os
import re
import collections
#from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
import itertools 
from nltk.tokenize import word_tokenize
from string import punctuation

from nltk import ngrams

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Functions

In [2]:
def read_texts(path):
    data = []
    file_name = os.listdir(path)

    for name in file_name:
        if name.endswith('.txt'):
            with open(path + name,encoding="utf8") as f:
                text = f.read()
                data.append({'nombre':name.replace('.txt',''), 'texto':text})

    df = pd.DataFrame(data)
    return df

In [3]:
# def filter_byindex(x,y):
#     #selectors = [x for x in col2]
#     return list(itertools.compress(x,y))

In [3]:
def check_len(x,y):
    if x==y:
        return 0
    else:
        return 1

In [4]:
def get_lf(i,j,k):
    if pd.isnull(k):
        regex_lf = re.compile(r'((?:\w+\W+){1,'+str(len(i))+'})\(\s'+i[0]+'.*\)')
        return regex_lf.findall(j)
    else:
        return k

In [5]:
def ngram_filter(doc, word, n):
    tokens = doc.split()
    all_ngrams = ngrams(tokens, n)
    filtered_ngrams = [x for x in all_ngrams if word in x]
    return filtered_ngrams

In [6]:
def get_longform(tokens, acro, margin = 2, i =1):
    long_form = ''
    #Looking for before
    for word in tokens[index-margin-len(acro):index]:
        #if first letter of word is equal to first letter os acronym
        if word[0] == acro[i].lower():
            long_form += word + ' '
            i += 1
            if i == len(acro):
                break
        elif (i == 1) and (word[0] == acro[i-1].lower()):
            long_form = word + ' '
            i = 1
            if i == len(acro):
                break
    long_form = long_form.rstrip()
    return long_form
    

# Load Data

### Testing

220 clinical cases.

In [7]:
testing_abbr = pd.read_csv("../datasets/testing_set/clinical_cases.abbreviations.testing_set.tsv", sep = '\t')
testing_met = pd.read_csv("../datasets/testing_set/clinical_cases.metadata.testing_set.tsv", sep = '\t')
testing_rel = pd.read_csv("../datasets/testing_set/clinical_cases.relations.testing_set.tsv", sep = '\t')

In [8]:
testing_met.head()

,# Document_ID,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1130-01082006000700014-1.txt,1.txt,1130-0108,2006-07-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-01082007000300006-7.txt,7.txt,1130-0108,2007-03-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S1134-80462009000100005-1.txt,1.txt,1134-8046,2009-02-01,Revista de la Sociedad Española del Dolor v.1...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S1137-66272014000300015-1.txt,1.txt,1137-6627,2014-12-01,Anales del Sistema Sanitario de Navarra v.37 ...,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0365-66912004001200011-1.txt,1.txt,0365-6691,2004-12-01,Archivos de la Sociedad Española de Oftalmolog...,http://scielo.isciii.es/scielo.php?script=sci_...


In [9]:
testing_rel = testing_rel.reset_index()

In [10]:
testing_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [11]:
testing_rel = testing_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [12]:
testing_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S0211-69952013000500019-1,SHORT_FORM,3739,ARA II,SHORT-LONG,LONG_FORM,3695,3737,antagonista del receptor de angiotesina II
1,S0211-69952013000500019-1,SHORT_FORM,2793,PCR,SHORT-LONG,LONG_FORM,2798,2837,reacción en cadena de enzima polimerasa
2,S0365-66912004000600008-1,SHORT_FORM,406,AV,SHORT-LONG,LONG_FORM,390,404,agudeza visual
3,S0211-69952012000500025-1,SHORT_FORM,945,angio-TAC,SHORT-LONG,LONG_FORM,908,943,angiotomografía computarizada axial
4,S1130-05582017000200122-1,SHORT_FORM,940,RMN,SHORT-LONG,LONG_FORM,910,938,resonancia magnética nuclear


In [13]:
testing_rel.Relation_type.unique()

array(['SHORT-LONG', 'SHORT-NESTED', 'NESTED-LONG'], dtype=object)

In [14]:
testing_abbr = testing_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [15]:
testing_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0004-06142010000500014-1,2037,2044,16SrRNA,16s ribosomal rna,16s ribosomal rno
1,S0004-06142010000500014-1,1349,1351,M.,mycobacterium,mycobacterium
2,S0004-06142010000500014-1,1339,1342,PCR,polymerase chain reaction,polymerase chain reaction
3,S0004-06142010000500014-1,611,615,BHCG,beta-human chorionic gonadotropin,beta-humar chorionic gonadotropin
4,S0004-06142010000500014-1,594,597,CEA,carcinoembrionary antigen,carcinoembrionary antiger


In [16]:
testing_raw = read_texts("../datasets/testing_set/testing_set.raw_text/")

In [17]:
testing_raw.head()

""


### Backgound

2879 clinical cases. 220 will be use for clinical evaluation

In [18]:
# back_met = pd.read_csv("../datasets/background_test/clinical_cases.metadata.background_set.tsv", sep = '\t')

FileNotFoundError: [Errno 2] File ../datasets/background_test/clinical_cases.metadata.background_set.tsv does not exist: '../datasets/background_test/clinical_cases.metadata.background_set.tsv'

In [ ]:
# back_met = back_met.rename(columns = {'# Document_ID': 'doc_id'})

In [ ]:
# back_met.head()

In [ ]:
# back_raw = read_texts("../datasets/background_test/background_test_set/")

In [ ]:
# back_raw.head()

### Development

146 clinical cases

In [19]:
dev_abbr = pd.read_csv("../datasets/development_set/clinical_cases.abbreviations.development_set.tsv", sep = '\t')
dev_met = pd.read_csv("../datasets/development_set/clinical_cases.metadata.development_set.tsv", sep = '\t')
dev_rel = pd.read_csv("../datasets/development_set/clinical_cases.relations.development_set.tsv", sep = '\t')

In [20]:
dev_met = dev_met.rename(columns = {'# Document_ID': 'doc_id'})

In [21]:
dev_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1130-14732005000300004-1.txt,1.txt,1130-1473,2005-06-01,Neurocirugía v.16 n.3 2005,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-01082008000200009-1.txt,1.txt,1130-0108,2008-02-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S1137-66272012000300021-1.txt,1.txt,1137-6627,2012-12-01,Anales del Sistema Sanitario de Navarra v.35 ...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S1699-695X2016000200010-1.txt,1.txt,1699-695X,2016-06-01,Revista Clínica de Medicina de Familia v.9 n....,http://scielo.isciii.es/scielo.php?script=sci_...
4,S1130-01082007001100009-1.txt,1.txt,1130-0108,2007-11-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...


In [22]:
dev_rel = dev_rel.reset_index()

In [23]:
dev_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [24]:
dev_rel = dev_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [25]:
dev_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S1888-75462015000400006-1,SHORT_FORM,1436,AP,SHORT-LONG,LONG_FORM,1419,1434,anteroposterior
1,S0210-56912009000700006-2,SHORT_FORM,876,angio-RMN,SHORT-LONG,LONG_FORM,858,874,angiorresonancia
2,S0210-56912009000700006-2,SHORT_FORM,819,RMN,SHORT-LONG,LONG_FORM,789,817,resonancia magnética nuclear
3,S1698-44472005000300015-1,SHORT_FORM,209,ATM,SHORT-LONG,LONG_FORM,177,207,articulación témporomandibular
4,S1698-44472005000300015-1,SHORT_FORM,1125,TC,SHORT-LONG,LONG_FORM,1099,1123,tomografía computerizada


In [26]:
dev_abbr = dev_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [27]:
dev_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S1130-14732005000300004-1,1216,1218,C3,tercera vértebra cervical,tercero vértebra cervical
1,S1130-14732005000300004-1,717,719,C2,segunda vértebra cervical,segundo vértebra cervical
2,S1130-14732005000300004-1,3191,3193,C3,tercera vértebra cervical,tercero vértebra cervical
3,S1130-14732005000300004-1,2867,2869,C3,tercera vértebra cervical,tercero vértebra cervical
4,S1130-14732005000300004-1,2862,2864,C2,segunda vértebra cervical,segundo vértebra cervical


In [28]:
dev_raw = read_texts("../datasets/development_set/development_set.raw_text/")

In [29]:
dev_raw.head()

""


### Sample test

15 clinical cases

In [30]:
sample_abbr = pd.read_csv("../datasets/sample_set/clinical_cases.abbreviations.sample_set.tsv", sep = '\t')
sample_met = pd.read_csv("../datasets/sample_set/clinical_cases.metadata.sample_set.tsv", sep = '\t')
sample_rel = pd.read_csv("../datasets/sample_set/clinical_cases.relations.sample_set.tsv", sep = '\t')

In [31]:
sample_met = sample_met.rename(columns = {'# Document_ID': 'doc_id'})

In [32]:
sample_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S0004-06142006000900015-1,1,0004-0614,2006-11-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
1,S0004-06142006000600015-1,1,0004-0614,2006-08-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S0004-06142007000700014-1,1,0004-0614,2007-09-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S0004-06142007000900013-1,1,0004-0614,2007-11-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0004-06142006000200014-1,1,0004-0614,2006-03-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...


In [33]:
sample_rel = sample_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [34]:
sample_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A_EndOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S0004-06142006000700014-1,SHORT_FORM,926,929,CEA,SHORT-LONG,LONG_FORM,896,924,Antígeno Carcino Embrionario
1,S0004-06142005001000011-1,SHORT_FORM,1626,1629,ROT,SHORT-LONG,LONG_FORM,1600,1624,reflejos osteotendinosos
2,S0004-06142005001000011-1,SHORT_FORM,1715,1718,RMN,SHORT-LONG,LONG_FORM,1685,1713,resonancia magnética nuclear
3,S0004-06142005001000011-1,SHORT_FORM,1663,1666,RCP,SHORT-LONG,LONG_FORM,1639,1661,reflejo cutaneoplantar
4,S0004-06142005001000011-1,SHORT_FORM,1808,1811,LCR,SHORT-LONG,LONG_FORM,1783,1806,líquido cefalorraquídeo


In [35]:
sample_abbr = sample_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [36]:
sample_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0004-06142005001000011-1,1034,1036,Kg,kilogramo,kilogramo
1,S0004-06142005001000011-1,1031,1033,mg,miligramo,miligramo
2,S0004-06142005001000011-1,196,199,IgA,inmunoglobulina a,inmunoglobulina a
3,S0004-06142005001000011-1,2057,2060,LCR,líquido cefalorraquídeo,líquido cefalorraquídeo
4,S0004-06142005001000011-1,1594,1598,EEII,extremidades inferiores,extremidad inferior


In [37]:
sample_raw = read_texts("../datasets/sample_set/sample_set.raw_text/")

In [38]:
sample_raw.head()

,nombre,texto
0,S0004-06142005001000011-1,Varón de 58 años de edad en el momento del tra...
1,S0004-06142005001000011-3,Mujer de 42 años en el momento de someterse a ...
2,S0004-06142005001000016-1,Varón de 65 años que consultaba por un síndrom...
3,S0004-06142006000200014-1,"Varón de 36 años, sin antecedentes personales ..."
4,S0004-06142006000300015-1,Paciente de 50 años con antecedente de litiasi...


### Trainning

318 clinical cases

In [39]:
train_abbr = pd.read_csv("../datasets/trainning_set/clinical_cases.abbreviations.training_set.tsv", sep = '\t')
train_met = pd.read_csv("../datasets/trainning_set/clinical_cases.metadata.training_set.tsv", sep = '\t')
train_rel = pd.read_csv("../datasets/trainning_set/clinical_cases.relations.training_set.tsv", sep = '\t')

In [40]:
train_met = train_met.rename(columns = {'# Document_ID': 'doc_id'})

In [41]:
train_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1139-76322015000500009-1.txt,1.txt,1139-7632,2015-12-01,Pediatría Atención Primaria v.17 n.68 2015,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-05582008000400007-2.txt,2.txt,1130-0558,2008-08-01,Revista Española de Cirugía Oral y Maxilofacia...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S0210-48062006000100012-1.txt,1.txt,0210-4806,2006-01-01,Actas Urológicas Españolas v.30 n.1 2006,http://scielo.isciii.es/scielo.php?script=sci_...
3,S0213-12852003000500002-1.txt,1.txt,0213-1285,2003-10-01,Avances en Odontoestomatología v.19 n.5 2003,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0212-71992005000400007-1.txt,1.txt,0212-7199,2005-04-01,Anales de Medicina Interna v.22 n.4 2005,http://scielo.isciii.es/scielo.php?script=sci_...


In [42]:
train_rel = train_rel.reset_index()

In [43]:
train_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [44]:
train_rel = train_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [45]:
train_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S1130-01082009000400014-1,SHORT_FORM,476,NPT,SHORT-LONG,LONG_FORM,454.0,474.0,nutrición parenteral
1,S1130-63432016000100009-1,SHORT_FORM,614,NIHSS,SHORT-LONG,LONG_FORM,621.0,662.0,National Institute of Health Stroke Scale
2,S1139-76322017000200007-1,SHORT_FORM,1145,CMV,SHORT-LONG,LONG_FORM,1128.0,1143.0,citomegalovirus
3,S1139-76322017000200007-1,SHORT_FORM,1243,VSG,SHORT-LONG,LONG_FORM,1206.0,1241.0,velocidad de sedimentación globular
4,S1139-76322017000200007-1,SHORT_FORM,1300,IGRA,SHORT-LONG,LONG_FORM,1267.0,1298.0,interferon-gamma release assays


In [46]:
train_abbr = train_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [52]:
# train_abbr.iloc[110:130]

In [48]:
train_abbr.Definition.nunique()

908

In [49]:
train_abbr[train_abbr['Abbreviation'] == '5-FU']

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
1214,S1130-63432013000100011-1,339,343,5-FU,5-fluorouracilo,5-fluorouracilo
1228,S1130-63432013000100011-1,2661,2665,5-FU,5-fluorouracilo,5-fluorouracilo
1231,S1130-63432013000100011-1,304,308,5-FU,5-fluorouracilo,5-fluorouracilo


In [53]:
# train_raw[train_raw['doc_id'] == 'S1137-66272012000300024-1'].iloc[0]['texto']

In [51]:
train_raw = read_texts("../datasets/trainning_set/training_set.raw_text/")

In [ ]:
train_raw = train_raw.rename(columns = {'nombre': 'doc_id'})

In [ ]:
train_raw.head()

# Sub-track 1

## Found abbreviations (Short Forms)

### Regex

In [ ]:
#patron = r'\(([A-Z]{2,8})\)'
#patron1 = r'\s[A-Z]{1,3}\s'
#patron2 = r'\s[a-z]{1,3}\s'
patron3 = r'[A-Z]{2,8}'
patron4 = r'\s[a-z]{1,2}\s'
patron5 = r'[a-z]+\-[A-Z]{1,8}'
patron6 = r'[a-z]+\/[a-z]+'
patron7 = r'[A-Z]?[a-z]{1,4}[A-Z]+[a-z]*[1-9]*'
patron8 = r'\/[a-z]*[A-Z]*'


# create a list with them
regexes = [ patron3, patron4, patron5, patron6, patron7,patron8]
for i in regexes:
    generic_re = re.compile("%s|%s|%s|%s|%s|%s" % (patron3, patron4, patron5, patron6, patron7,patron8))

In [ ]:
#nltk.download('stopwords')
swords = list(set(stopwords.words('spanish')))

**Get Short Formns with a regex in each text**

In [ ]:
train_raw['abrev'] = train_raw['texto'].map(lambda x: generic_re.findall(x))

In [ ]:
# train_raw['offse'] = train_raw['texto'].map(lambda x: generic_re.search(x).span())

**Get offsets of the Short Forms founded in the text**

In [ ]:
train_raw['offse'] = train_raw['texto'].map(lambda x: [(m.start(0), m.end(0)) for m in re.finditer(generic_re, x)])

In [ ]:
train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [i.strip() for i in x])

Filter Short Forms to delete ones which are stopwords, get their index too for filter offsets lists later

In [ ]:
train_raw['abrev_index'] = train_raw['abrev'].apply(lambda x: [x.index(i) for i in x if i not in swords])

In [ ]:
train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [i for i in x if i not in swords])

Filter offsets lists by index

In [ ]:
train_raw['offse'] = train_raw.apply(lambda x: [x['offse'][i] for i in x['abrev_index']], axis = 1)

In [ ]:
train_raw.head()

In [ ]:
train_raw[train_raw['doc_id'] == 'S0004-06142005001000011-1']['abrev']

Check abrev and offse columns has the same length

In [ ]:
train_raw['abrev_len'] =train_raw['abrev'].str.len()
train_raw['offse_len'] =train_raw['offse'].str.len()

In [ ]:
train_raw.head()

Check both lists have the same lenght

In [ ]:
train_raw['len_check'] = train_raw.apply(lambda row: check_len(row['abrev_len'],row['offse_len']),axis = 1)

Delete rows with different lenghts (check it later)

In [ ]:
train_raw.shape

In [ ]:
train_raw[train_raw['len_check'] != 1].shape

In [ ]:
#train_raw = train_raw[train_raw['len_check'] != 1]

In [ ]:
train_raw.shape

**Remove stopwords from text**

In [ ]:
train_raw['tokens'] = train_raw['texto'].map(lambda x: word_tokenize(x))
train_raw['texto_clean'] = train_raw['tokens'].map(lambda x: ' '.join([w for w in x if w not in swords]))

In [ ]:
train_raw.head()

Remove punctuation from abreviations

In [ ]:
#train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [a.split("/") for a in x])

In [ ]:
#train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [item for sublist in x for item in sublist])

In [ ]:
for i in punctuation:
    train_raw['abrev'] = train_raw[train_raw['abrev'].notnull()]['abrev'].apply(lambda x: [a.replace(i,"") for a in x])

In [ ]:
train_raw['zip'] = train_raw.apply(lambda row: list(zip(row['abrev'], row['offse'])), axis = 1)

In [ ]:
train_raw.head()

**Get one row per abbreviation**

Separate elements lists in different rows

In [ ]:
mine = train_raw.explode('zip')
#mine = mine.explode('offse')

In [ ]:
mine.shape

In [ ]:
mine[mine['zip'].isnull()].shape

In [ ]:
mine = mine[['doc_id', 'texto_clean', 'tokens', 'zip']]

In [ ]:
mine = mine[mine['zip'].notnull()]

In [ ]:
mine.head()

In [ ]:
mine['abrev'] = mine.apply(lambda row: row['zip'][0], axis = 1)

In [ ]:
mine['offsets'] = mine.apply(lambda row: row['zip'][1], axis = 1)

In [ ]:
mine = mine[mine['abrev'] != ""]

In [ ]:
mine.head()

Separate offsets tuples in different columns

In [ ]:
mine[['startOffset', 'endOffset']] = pd.DataFrame(mine['offsets'].tolist(), index=mine.index) 

In [ ]:
mine = mine[['doc_id', 'texto_clean', 'tokens', 'abrev', 'startOffset', 'endOffset']]

Delete null values, and change ttype to integer for offsets columns

In [ ]:
mine.shape

In [ ]:
mine.dropna(subset=['startOffset', 'endOffset'], inplace = True)

In [ ]:
mine.shape

In [ ]:
mine['startOffset'] = mine['startOffset'].astype(int)
mine['endOffset'] = mine['endOffset'].astype(int)

In [ ]:
mine = mine[['doc_id', 'texto_clean', 'tokens', 'abrev', 'startOffset', 'endOffset']]

In [ ]:
mine.shape

In [ ]:
mine = mine.drop_duplicates(subset = ['doc_id', 'texto_clean', 'abrev', 'startOffset', 'endOffset'])

In [ ]:
mine.shape

In [ ]:
mine.head()

Check dataframes for one text

In [ ]:
# mine[mine['doc_id'] == 'S1130-05582012000300005-1']

In [ ]:
# train_abbr[train_abbr['doc_id'] == 'S1130-05582012000300005-1']

## Search Long Formns

### Search Long Forms in the same text

**measurement units dictionary**

In [ ]:
# mu_dic = {"mL":"Mililitro",
# "mg":"Miligramo",
# "g":"Gramo",
# "L":"Litro",
# "mcg":"Microgramo",
# "mmol":"Milimol",
# "UI":"Unidades Internacionales",
# "Miles UI":"Miles de Unidades Internacionales",
# "Millones UI":"Millones de Unidades Internacionales",
# "UFC":"Unidades Formadoras de Colonias",
# "mEq":"Miliequivalente",
# "ng":"Nanogramo",
# "Lf":"Unidad Floculante",
# "UFP":"Unidad Formadora de Placa",
# "DIC":"Dosis Infectante Mediana de Cultivo Celular 50% ",
# "DIT":"Dosis Infectante Mediana de Cultivo Tisular 50% ",
# "DI":"Dosis Infectante 50% ",
# "mol":"Peso Molecular Gramo ",
# "Eq":"Peso Equivalente Gramo ",
# "Dosis":"Dosis",
# "Almh":"Almohadilla",
# "Amp":"Ampolla",
# "Anl":"Anillo",
# "Bar":"Barra",
# "Bolsa":"Bolsa",
# "Cap":"Capsula",
# "Car":"Caramelo",
# "Carp":"Carpula",
# "Cart":"Cartucho",
# "Com":"Comprimido",
# "Dia":"Dia",
# "Fras":"Frasco",
# "Fras-Amp":"Frasco Ampolla ",
# "Grag":"Gragea",
# "Hora":"Hora",
# "Imp":"Implante",
# "Jab":"Jab¢n",
# "Jer":"Jeringa Prellenada ",
# "uL":"Microlitro",
# "Ovu":"Ovulo",
# "Parche":"Parche",
# "Past":"Pastilla",
# "Perl":"Perla",
# "Pil":"Pildora",
# "Pip":"Pipeta",
# "%":"Porcentaje",
# "Sach":"Sachet",
# "Sob":"Sobre",
# "Sup":"Supositorio",
# "Tab":"Tableta",
# "Troc":"Trocisco",
# "Vial":"Vial",
# "Kg":"Kilogramo",
# "Gal":"Galon",
# "Sis":"Sistema Terapeutico",
# "mCi":"miliCuries",
# "mBq":"milibequerel",
# "UEL":"Unidades ELISA ",
# "DL":"Dosis Letal",
# "U USP":"Unidades USP ",
# "U":"Unidades",
# "Rot":"Rotacaps",
# "CCID":"Dosis Infecciosa en Cultivo de Célula ",
# "U":"UNIDAD",
# "Otros":"Otros",
# "µ Ci":"mcroCuries",
# "Esp":"Esporas",
# "mcHA":"microgramos de HA ",
# "Gom":"Goma",
# "KIU":"Unidad Inhibidora de Calicreina ",
# "mcel":"Millones de Células ",
# "DU":"Unidades de Antigeno D",
# "Dil D2":"Dil D2",
# "Tin.Mad.":"Tintura Madre",
# "Dil D4":"Dil D4",
# "Dil D5":"Dil D5",
# "Dil D1":"Dil D1",
# "Dil D8":"Dil D8",
# "Dil D3":"Dil D3",
# "OU":"Unidad de Opacidad ",
# "mm": "milimetro",
# "dm": "decimetro",
# "cm": "centimetro"}

In [ ]:
# mine['long_form'] = mine['abrev'].map(mu_dic)

In [ ]:
mine_lf = mine[mine['abrev'].str.len() > 1].reset_index()

**With regex looking for words before SF**

In [ ]:
def get_longform(tokens, acro, margin = 2, i =0):
    index = tokens.index(acro)
    long_form = ''
    #Looking for before
    for word in tokens[index-margin-len(acro):index]:
        #if first letter of word is equal to first letter os acronym
        if word[0] == acro[i].lower():
            long_form += word + ' '
            i += 1
            if i == len(acro):
                break
        elif (i == 1) and (word[0] == acro[i-1].lower()):
            long_form = word + ' '
            i = 1
            if i == len(acro):
                break
    long_form = long_form.rstrip()
    return long_form

In [ ]:
mine['long_form'] = mine.apply(lambda row: get_longform(row['tokens'], row['abrev']), axis = 1)

In [ ]:
train_raw['tokens'] = train_raw['texto_clean'].map(lambda x: word_tokenize(x))

In [ ]:
prueba = mine[mine['doc_id']=='S0004-06142005001000011-1']

In [ ]:
for i in punctuation:
    prueba['tokens'] = prueba['tokens'].apply(lambda x: [a.replace(i,"") for a in x])

In [ ]:
prueba

In [ ]:
prueba.apply(lambda row: get_longform(row['tokens'], row['abrev']), axis = 1)

In [ ]:
get_longform(mine[mine['doc_id'] == 'S0004-06142005001000011-1'].iloc[3]['tokens'], 'PCR')

Check how many texts don't have LF for the SF in the same text

In [ ]:
# mine['long_form'] = mine['long_form'].map(lambda x: x.replace(' ',''))

In [ ]:
mine_lf[(mine_lf['abrev'].notnull()) & (mine_lf['long_form']== '')]

In [ ]:
lf_null = mine_lf[(mine_lf['abrev'].notnull()) & (mine_lf['long_form'] == '')].shape[0]

In [ ]:
print(f"LF has not be found in the same text where the SF is in {lf_null/mine_lf.shape[0]*100: .2f}% of texts")

**Create dictionary with SF and LF pairs founded in the same text**

In [ ]:
df_pairs = mine_lf[(mine_lf['abrev'].notnull()) & (mine_lf['long_form'] != '')]

In [ ]:
df_pairs.head()

In [ ]:
pairs_dic = {}
for index, row in df_pairs.iterrows():
    if not row['abrev'] in pairs_dic:
        pairs_dic[row['abrev']] = set()
    pairs_dic[row['abrev']].add(row['long_form'])

In [ ]:
pairs_dic

In [ ]:
len(pairs_dic.keys())

## Using N-grams

In [ ]:
for i in punctuation:
    mine['texto_clean'] = mine['texto_clean'].map(lambda x: x.replace(i,""))

In [ ]:
a = mine.head(10)

In [ ]:
a.apply(lambda row: ngram_filter(row['texto_clean'], row['abrev'], len(row['abrev'])+1), axis = 1)[2]

In [ ]:
mine['lf_ngrams'] = mine.apply(lambda row: ngram_filter(row['texto_clean'], row['abrev'], len(row['abrev'])+1), axis = 1)

In [ ]:
mine.head()

In [ ]:
from nltk import ngrams

sentence = 'this is a foo bar sentences and i want to ngramize it this is it and'

n = 6
sixgrams = ngrams(sentence.split(), 2)

for grams in sixgrams:
    esBigramFreq = collections.Counter(grams)
    print(grams,esBigramFreq , esBigramFreq.most_common(10))

**Get context vectors**

1. Bag of words model

In [ ]:
count_vect = CountVectorizer()

In [ ]:
text_counts = count_vect.fit_transform(mine['texto_clean'])

In [ ]:
text_counts.shape

In [ ]:
tfidf_transformer = TfidfTransformer()
texto_tfidf = tfidf_transformer.fit_transform(text_counts)
texto_tfidf.shape

In [ ]:
from keras.preprocessing.text import Tokenizer

sentence = ["John likes to watch movies. Mary likes movies too."]

def print_bow(sentence: str) -> None:
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentence)
    sequences = tokenizer.texts_to_sequences(sentence)
    word_index = tokenizer.word_index
    print(sequences,word_index)
    bow = {}
    for key in word_index:
        bow[key] = sequences[0].count(word_index[key])
    print(bow)
    print(f"Bag of word sentence 1 :\n{bow}")
    print(f'We found {len(word_index)} unique tokens.')

print_bow(sentence)